# Failure Prediction PJT

## Machine Learning 프로젝트 수행을 위한 코드 구조화
(분류, 회귀 Task)

### 템플릿 코드 생성 for ML Project
1. 필요한 라이브러리와 데이터를 불러옵니다.


2. EDA를 수행합니다. 이 때, EDA의 목적은 풀어야하는 문제를 위해서 수행됩니다.


3. 전처리를 수행합니다. 이 때, 중요한 것은 feature engineering을 어떻게 하느냐 입니다.


4. 데이터 분할을 진행합니다. 이 때, train data와 test tada 간의 분포 차이가 없는 지 확인합니다.


5. 학습을 진행합니다. 어떤 모델을 사용하여 학습할지 정합니다. 성능이 잘 나오는 GBM을 추천합니다.


6. Hyper-parameter tuning을 수행합니다. 원흐는 목표 성능이 나올 때 까지 진행합니다. 
   검증 단계를 통해, 지속적으로 overfitting이 되지 않게 주의하세요.


7. 최종 테스트를 진행합니다. 데이터 대회 포맷에 맞는 submission 파일을 만들어서 성능을 확인해보세요.



## 1. 라이브러리, 데이터 불러오기

In [2]:
# 데이터분석 4종 세트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 모델들, 성능 평가
# 일반적으로 정형데이터로 머신러닝 분석을 진행할 때는 아래 2개 모델을 돌려봅니다. 특히 RandomForest가 테스트하기 좋습니다.
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from lightgbm.sklearn import LGBMClassifier
from lightgbm.sklearn import LGBMRegressor

# 상관관계 분석, VIF : 다중공선성 제거
from statsmodels.stats.outliers_influence import variance_inflation_factor

# KFold(CV), partial : optuna를 사용하기 위함
from sklearn.model_selection import KFold
from functools import partial

# hyper-parameter tuning을 위한 라이브러리, optuna
import optuna

In [3]:
# flag setting
data_reducing = False ## memory reducing technique
feature_reducing = 'fi' ## 'pca' or 'fi'
online = False ## use online training
sampling = 'hybrid' ## 'under' or 'over' or 'hybrid'

In [8]:
# 데이터를 불러옵니다.

# Related kaggle competitions
# Name: Bosch Production Line Performance
# Link: https://www.kaggle.com/competitions/bosch-production-line-performance

# train_cat_part = pd.read_csv('../input/bosch-production-line-performance/train_categorical.csv.zip',
#                             nrows=1000)
train_num_part = pd.read_csv('./bosch-production-line-performance/train_numeric.csv.zip',
                            nrows=1000)
train_date_part = pd.read_csv('./bosch-production-line-performance/train_date.csv.zip',
                             nrows=1000)

#print(train_cat_part.shape, train_num_part.shape, train_date_part.shape)
print(train_num_part.shape, train_date_part.shape)

(1000, 970) (1000, 1157)


In [ ]:
# categorical data는 EDA 결과 분석에 큰 도움이 없어, 사용하지 않겠습니다.

# total = train_cat_part.count() * 0
# N = 0

# for chunk in pd.read_csv('../input/bosch-production-line-performance/train_categorical.csv.zip',
#                         chunksize=100000):  # chunksize가 클수록, 속도는 빨라지고 메모리 사용량이 늘어납니다.
    
#     total = total + chunk.isnull().sum()  # 전체 데이터의 column별 결측치 개수
#     N = N + len(chunk)  # 전체 데이터의 row 수
    
# display(total)
# print(N)
# usecols = total.index[(total / N) < 0.5] # train_categorical.csv에 있는 결측치가 50% 미만인 column들.
# usecols

In [5]:
# train_cat = pd.read_csv('../input/bosch-production-line-performance/train_categorical.csv.zip',
#                        usecols=usecols)
# train_cat

train_date.csv에서는 파생 변수를 만들어서 사용합니다.


1. 각 station별로 측정 시간이 모두 있거나, 없거나의 케이스만 존재한다.


- 데이터 중에서 station별로 하나의 column만 뽑아서 체크해도 괜찮다.

<br>


2. 각 station별로 시간이 모두 같지는 않다. 중간에 측정시간이 바뀌는 케이스도 있다.

- 데이터 별로 min, max값을 계산하면 된다.


<br>


- start_station : 해당 product가 처음으로 공정을 거친 station_name


- end_station : 해당 product가 마지막으로 공정을 거친 station_name


- time_diff : 해당 product가 처음 시작한 timestamp와 마지막으로 끝낸 timestamp의 차이 (공정에 걸린 시간)

In [ ]:
# 1157 cols -> station 개수(S0 ~ S51)
# 1. column들이 나와있는 series를 하나 만듭니다.
train_date_part.count()
# 2. 그 series의 index를 뽑아서 가운데 _S0_ 를 뽑습니다.
train_date_part.count().reset_index()['index'].str.split('_', expand=True)[1]
# 3. 각 station별로 하나씩만 남깁니다. + Id column 제거
train_date_part.count().reset_index()['index'].str.split('_', expand=True)[1].drop_duplicates()[1:]
# 4. 각 station별 index에 해당하는 column들을 뽑습니다.
temp = train_date_part.count().reset_index()['index'].str.split('_', expand=True)[1].drop_duplicates()[1:].index
datecols = train_date_part.columns[temp]
datecols

In [ ]:
# train_date_part 데이터를 가지고, 원하는 3개의 파생변수가 잘 생성되는지 확인합니다.

In [ ]:
train_date_part["start_station"] = -1  # 초기화(initialization)
train_date_part["end_station"] = -1

for col in train_date_part[datecols]:
    ## TODO : start_station과 end_station을 찾아주세요
    station_name = col.split("_")[1] # S0
    station_num = int(col.split("_")[1][1:]) # 0
    notnulls = (~train_date_part[col].isnull()) # 해당 column에서 NaN 아닌 값들의 위치 (boolean mask)
    
    # NaN이라면 해당 station을 지나간 것. 아니라면 해당 station에서 측정이 된 것.
    # start_station은 해당 column에서 "처음으로" 수치값이 등장해야한다.(NaN이 아님) 즉, start_station이 -1이며 notnull이어야 함.
    # end_station은 해당 column에 수치값이 등장하면 매번 update한다. 즉, notnull이기만 하면 됨.
    train_date_part.loc[(notnulls & (train_date_part.start_station == -1)), "start_station"] = station_num
    train_date_part.loc[notnulls, "end_station"] = station_num

start_time = train_date_part.drop(columns=["Id", "start_station", "end_station"]).min(axis=1) # start_time
end_time = train_date_part.drop(columns=["Id", "start_station", "end_station"]).max(axis=1) # end_time
time_diff = end_time - start_time  # 공정에 걸린 시간.
train_date_part['time_diff'] = time_diff
train_date = train_date_part[["Id", "start_station", "end_station", "time_diff"]]
train_date

In [ ]:
## TO-DO: train_date.csv를 불러와서, start_station, end_station, time_diff 파생 변수를 생성한 뒤에,
##        1183747 x 4 에 해당하는 train_date DataFrame을 만들어주세요.

## [HINT]
# 1. chunk (datecols)
# 2. start, end station 만들었던거 그대로 사용
# 3. min, max 계산해서 time_diff 구하기
extract_cols = ["Id", "start_station", "end_station", "time_diff"]
train_date = pd.DataFrame(columns=extract_cols)

for chunk in pd.read_csv('../input/bosch-production-line-performance/train_date.csv.zip',
                        usecols=["Id"] + datecols.tolist(), chunksize=100000):

    chunk["start_station"] = -1  # 초기화(initialization)
    chunk["end_station"] = -1

    for col in chunk[datecols]: # Id, start_station, end_station 제외한것과 같음.
        ## TODO : start_station과 end_station을 찾아주세요
        station_num = int(col.split("_")[1][1:])
        notnulls = (~chunk[col].isnull()) # 해당 column에서 NaN 아닌 값들의 위치 (boolean mask)

        chunk.loc[(notnulls & (chunk.start_station == -1)), "start_station"] = station_num
        chunk.loc[notnulls, "end_station"] = station_num
        
        
    start_time = chunk[datecols].min(axis=1)
    end_time = chunk[datecols].max(axis=1)
    time_diff = end_time - start_time
    chunk['time_diff'] = time_diff
    chunk = chunk[extract_cols]
    train_date = pd.concat([train_date, chunk])
    
train_date

In [ ]:
# train_numeric.csv 데이터에서 중요한 feature를 뽑기 위해서, PCA(truncatedSVD) 또는 feature importance를 통해 뽑는 방법을 사용합니다.

#### [WARNING] 이 데이터는 1000개의 데이터에 대해서만 계산한 값으로, 제대로 하려면 전체 데이터를 불러와서 계산해야 합니다!

In [ ]:
# train_num.csv에서 결측치가 70% 이하인 데이터.
nullcols = train_num_part.columns[train_num_part.isnull().mean() < 0.7]
nullcols

In [ ]:
if feature_reducing == 'pca':
    from sklearn.decomposition import TruncatedSVD ## for sparse input

    train_num = pd.read_csv('../input/bosch-production-line-performance/train_numeric.csv.zip',
                           usecols=nullcols)

    X = train_num.drop(["Id", "Response"], axis=1).fillna(0)  # 1183747 x 186 features
    svd = TruncatedSVD(n_components=15, n_iter=30, random_state=42) ## more than 300 iterations!
    X_svd = svd.fit_transform(X)
    print(X_svd.shape) # 186 features -> 15 features.

In [6]:
# SVD로 만든 numpy array를 보기 좋게 DataFrame으로 변환합니다.

In [ ]:
if feature_reducing == 'pca':
    sv_cols = [f"SV{i}" for i in range(1, X_svd.shape[1]+1)]
    df_svd = pd.DataFrame(data=X_svd, columns=sv_cols)
    df_svd["Id"] = train_num["Id"] # Id column 추가
    df_svd["Response"] = train_num["Response"] # target value 추가
    display(df_svd)
    
    # release memory
    del train_num
    del X
    del X_svd

    gc.collect()
    
    # train_date에서 추출한 정보와, train_numeric에서 추출한 정보를 합쳐줍니다.
    X_train = pd.merge(df_svd, train_date, on="Id")
    display(X_train)

In [ ]:
# RandomForestClassifier를 이용해서 feature importance가 높은 변수 추출하기

In [ ]:
if feature_reducing == 'fi' and False:  ## 현재 버젼에서는 training을 안시키려고 강제로 False로 만듬.
    # Online learning (= batch training)
    if online:
        from tqdm.notebook import tqdm

        clf = RandomForestClassifier(n_estimators=50, max_depth=31, max_features=0.7, n_jobs=-1,
                                    random_state=42)#, warm_start=True)

        cnt = 0
        for chunk in pd.read_csv("../input/bosch-production-line-performance/train_numeric.csv.zip",
                                chunksize=100000):  # each iteration, training with 50000 rows
            cnt += 1
            X = chunk.drop(["Id", "Response"], axis=1).fillna(0)  # 10000 x 978 features
            y = chunk.Response
            clf.fit(X, y)
            print(f"in {cnt} iterations.")
     
    # offline learning(=full-batch training)
    else:
        train_num = pd.read_csv('../input/bosch-production-line-performance/train_numeric.csv.zip',
                               nrows=100000)

        X = train_num.drop(["Id", "Response"], axis=1).fillna(0)  # 10000 x 978 features
        y = train_num.Response

        # max_features : sqrt(# of features) ~ 31
        clf = RandomForestClassifier(n_estimators=50, max_depth=31, max_features=0.7, n_jobs=-1,
                                    random_state=42)
        clf.fit(X, y)
        
        feature_importances = clf.feature_importances_
        fi_index = feature_importances.argsort()[::-1][:15] # feature_importance가 높은 순서대로의 index
        usecols = train_num.columns[fi_index]
        print("Features from RF: ", usecols)


In [7]:
# Using know Numeric Feature from below github link
# https://github.com/aakashveera/bosch-production-line-performance/blob/master/Numeric%20Classifier.ipynb